In [21]:
import requests
from dotenv import load_dotenv
import json
import pandas as pd
from pathlib import Path
import calendar

### Quarterly GDP

In [57]:

url = 'https://www.alphavantage.co/query?function=REAL_GDP&interval=quarterly&apikey=ALPA_VANTANGE_API_key'
url_format = url + '?format=json'
response = requests.get(url_format).json()
gdp = pd.DataFrame(response)
# Split 'data' column into 'date' and 'value' columns
gdp[['date', 'value']] = gdp['data'].apply(lambda x: pd.Series([x['date'], x['value']]))

# Drop the original 'data' column & unnecessary columns
gdp = gdp.drop(columns={'name', 'interval', 'unit', 'data'})
gdp = gdp.rename(columns={'date': 'Date', 'value': 'GDP'}).set_index('Date')
gdp.head()

,GDP
Date,
2023-01-01,4949.655
2022-10-01,5157.178
2022-07-01,5018.093
2022-04-01,4985.795
2022-01-01,4855.857


In [58]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 2023-01-01 to 2002-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   GDP     85 non-null     object
dtypes: object(1)
memory usage: 1.3+ KB


In [59]:
gdp_df = gdp.loc['2023-01-01':'2020-01-01']
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 2023-01-01 to 2020-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   GDP     13 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


### Quarterly GDP Per Capita

In [67]:
url = 'https://www.alphavantage.co/query?function=REAL_GDP_PER_CAPITA&apikey=ALPA_VANTANGE_API_key'
url_format = url + '?format=json'
response = requests.get(url_format).json()
gdp_per_capita = pd.DataFrame(response)
# Split 'data' column into 'date' and 'value' columns
gdp_per_capita[['date', 'value']] = gdp_per_capita['data'].apply(lambda x: pd.Series([x['date'], x['value']]))

# Drop the original 'data' column & unnecessary columns
gdp_per_capita = gdp_per_capita.drop(columns={'name', 'interval', 'unit', 'data'})
gdp_per_capita = gdp_per_capita.rename(columns={'date': 'Date', 'value': 'GDP PER CAPITA'}).set_index('Date')
gdp_per_capita.head()

,GDP PER CAPITA
Date,
2023-01-01,60502.0
2022-10-01,60376.0
2022-07-01,60080.0
2022-04-01,59688.0
2022-01-01,59836.0


In [62]:
gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305 entries, 2023-01-01 to 1947-01-01
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   GDP PER CAPITA  305 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [64]:
gdp_capita_df = gdp_per_capita.loc['2023-01-01':'2020-01-01']
gdp_capita_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 2023-01-01 to 2020-01-01
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   GDP PER CAPITA  13 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


### DOL Monthly report

In [65]:
url = 'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=ALPA_VANTANGE_API_key'
url_format = url + '?format=json'
response = requests.get(url_format).json()
labor = pd.DataFrame(response)
# Split 'data' column into 'date' and 'value' columns
labor[['date', 'value']] = labor['data'].apply(lambda x: pd.Series([x['date'], x['value']]))

# Drop the original 'data' column & unnecessary columns
labor = labor.drop(columns={'name', 'interval', 'unit', 'data'})
labor = labor.rename(columns={'date': 'Date', 'value': 'DOL'}).set_index('Date')
labor.head()

,DOL
Date,
2023-05-01,3.7
2023-04-01,3.4
2023-03-01,3.5
2023-02-01,3.6
2023-01-01,3.4


In [66]:
labor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 2023-05-01 to 1948-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DOL     905 non-null    object
dtypes: object(1)
memory usage: 14.1+ KB


In [ ]:
labor_df = labor.loc['2023-']

### Consumer Pricing Index CPI Monthly Report

In [47]:
# read json report in csv format
with open('./cpi_report.json') as json_file:
    cpi_data = json.load(json_file)
cpi_df = pd.json_normalize(cpi_data['Results']['series'][0]['data'])
cpi_df.drop(columns=['period', 'latest', 'footnotes'], axis=1, inplace=True)
cpi_df.head()

,year,periodName,value
0,2023,April,303.363
1,2023,March,301.836
2,2023,February,300.840
3,2023,January,299.170
4,2022,December,296.797


In [48]:
# integer encoding of `periodName` 
name_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
cpi_df['month'] = cpi_df['periodName'].apply(lambda x: name_to_num[x])
cpi_df.drop(['periodName'], inplace=True, axis=1)
cpi_df.head()

,year,value,month
0,2023,303.363,4
1,2023,301.836,3
2,2023,300.840,2
3,2023,299.170,1
4,2022,296.797,12


In [50]:
# Convert 'year' and 'month' columns to strings
cpi_df['year'] = cpi_df['year'].astype(str)
cpi_df['month'] = cpi_df['month'].astype(str)

# Create a new column 'date' by combining 'year' and 'month' columns
cpi_df['Date'] = cpi_df['year'] + '-' + cpi_df['month'].str.zfill(2) + '-01'
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
# drop the original `year` & `month` columns
if 'year' in cpi_df.columns and 'month' in cpi_df.columns:
    cpi_df.drop(['year', 'month'], axis=1, inplace=True)
cpi_df.set_index('Date', inplace=True)
cpi_df.head()

,value
Date,
2023-04-01,303.363
2023-03-01,301.836
2023-02-01,300.840
2023-01-01,299.170
2022-12-01,296.797


In [45]:
cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40 entries, 2023-04-01 to 2020-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   value   40 non-null     object
dtypes: object(1)
memory usage: 640.0+ bytes
